In [1]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir
from os.path import join
from sklearn import metrics
import os

In [2]:
local_dir = os.path.join("/DATA",os.environ.get("USER"),"MNIST_data")
os.makedirs(local_dir,mode=0o755, exist_ok=True)

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(local_dir, one_hot=True)

Extracting /DATA/solomiia.kurchaba/MNIST_data/train-images-idx3-ubyte.gz
Extracting /DATA/solomiia.kurchaba/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /DATA/solomiia.kurchaba/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /DATA/solomiia.kurchaba/MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
num_input = 28 
time_steps = 28 
hidden = 128
output_classes = 10
lr=0.0001
batch_size=128

In [5]:
x_tr, y_batches = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
x_batches = x_tr.reshape(-1, time_steps, num_input)
x_t=mnist.test.images
x_test=x_t.reshape(-1,time_steps,num_input)
y_test=mnist.test.labels

# Sieć RNN

In [10]:
from tensorflow.contrib import rnn
def network(rs, num_input = 28,time_steps = 28, hidden = 128, output_classes = 10, lr=0.0001,batch_size=128,epochs=1000):
    tf.reset_default_graph()
    X=tf.placeholder(tf.float32,[None,time_steps,num_input])
    Y=tf.placeholder(tf.float32,[None,output_classes])
    x=tf.unstack(X,time_steps,1)
    tf.set_random_seed(rs)
    basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=hidden) # here the first set of weights and biases is defined
    rnn_output, states=rnn.static_rnn(basic_cell,x,dtype=tf.float32)
    stacked_rnn_output=tf.reshape(rnn_output[-1],[-1,hidden])
    stacked_outputs=tf.layers.dense(stacked_rnn_output,output_classes)# the output layer. (+additional set of weights and biases)
    predict=tf.nn.softmax(stacked_outputs)
    y_pred=tf.argmax(predict,1)
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predict, labels=Y))
    optimizer=tf.train.AdamOptimizer(learning_rate=lr)
    training_op=optimizer.minimize(loss_op)
    correct_pred = tf.equal(tf.argmax(predict, 1), tf.argmax(Y, 1))
    true_ans=tf.cast(correct_pred,tf.float32)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    init=tf.global_variables_initializer()
    with  tf.Session() as sess:
        init.run()
        for ep in range(epochs):
            sess.run(training_op,feed_dict={X: x_batches, Y: y_batches})
            loss=loss_op.eval(feed_dict={X: x_batches, Y: y_batches})
            if ep % 100 == 0:
                loss=loss_op.eval(feed_dict={X: x_batches, Y: y_batches})
                print(ep,"\tloss:",loss)
        train_accuracy=accuracy.eval(feed_dict={X: x_batches, Y: y_batches})  
        print(train_accuracy)
        test_accuracy=accuracy.eval(feed_dict={X:x_test, Y:y_test}) 
        print(test_accuracy)
        true_ans=true_ans.eval(feed_dict={X: x_test, Y: y_test})
        
        
        
    sess.close()   
    return true_ans
    

In [ ]:
# Sprawdzamy czy przy różnych startowych parametrach, sieć robi błędy na tych samych obrazkach
T=[]
for i in range (10):
    true=network(i)
    T.append(true)

    

In [183]:
true=pd.DataFrame(T)
true=true.T
true
# wyniki eksperymentu wskazuja na to, że sieć myli się w różnych miejscach, w zależności od początkowej inicializacji parametrów,chociaz msą obrazki na których mylą się zawsze

,0,1,2,3,4,5,6,7,8,9
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
2,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
3,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0


In [52]:
rnn=network(1)

0 	loss: 2.29871
100 	loss: 1.99783
200 	loss: 1.77619
300 	loss: 1.62369
400 	loss: 1.5681
500 	loss: 1.55679
600 	loss: 1.50414
700 	loss: 1.49697
800 	loss: 1.4949
900 	loss: 1.49385
0.96875
0.5616


In [101]:
rnn

array([ 1.,  1.,  0., ...,  1.,  0.,  1.], dtype=float32)

# Convolutional network

In [13]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [79]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
x_tr, y_batches = mnist.train.next_batch(150)

In [105]:
def Convolution(rs,inp=784,out=10,epochs=1000, lr=1e-4):
    x = tf.placeholder(tf.float32, shape=[None, inp])
    y_ = tf.placeholder(tf.float32, shape=[None, out])
    tf.set_random_seed(rs)
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    x_image = tf.reshape(x, [-1,28,28,1])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    W_fc2 = weight_variable([1024, out])
    b_fc2 = bias_variable([out])
    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    true_ans=tf.cast(correct_prediction,tf.float32)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    init=tf.global_variables_initializer()
    with  tf.Session() as sess:
        init.run()
        for i in range(epochs):
            batch = mnist.train.next_batch(150)
            if i%100 == 0:
                train_accuracy = accuracy.eval(feed_dict={ x:batch[0], y_: batch[1], keep_prob: 0.8})
                print("step %d, training accuracy %g"%(i, train_accuracy))
    
            train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.8})

        print("test accuracy %g"%accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
        corr=true_ans.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})
    sess.close()
    return corr
    

In [83]:
x_tr, y_batches = mnist.train.next_batch(128)

In [148]:
c=Convolution(1)

step 0, training accuracy 0.0933333
step 100, training accuracy 0.786667
step 200, training accuracy 0.873333
step 300, training accuracy 0.906667
step 400, training accuracy 0.96
step 500, training accuracy 0.926667
step 600, training accuracy 0.953333
step 700, training accuracy 0.973333
step 800, training accuracy 0.966667
step 900, training accuracy 0.966667
test accuracy 0.9776


In [ ]:
C=[]
for i in range(10):
    cc=Convolution(i)
    C.append(cc)

In [176]:
np.nonzero(C[0]<1)
N=[]
for i in range (10):
    nonz=np.nonzero(C[i]<1)
    N.append(nonz)


In [185]:
#Widzimy że sieci zinicializowane z różnych losowych stanów mają wspólne obrazki na których się mylą
np.intersect1d(N[0],N[5])

array([ 247,  259,  290,  320,  321,  447,  582,  591,  659,  684,  720,
        740,  844,  947, 1014, 1039, 1062, 1107, 1112, 1226, 1232, 1247,
       1260, 1299, 1319, 1326, 1393, 1500, 1527, 1530, 1553, 1621, 1678,
       1681, 1709, 1754, 1790, 1800, 1850, 1878, 1901, 2016, 2043, 2053,
       2098, 2109, 2129, 2130, 2182, 2189, 2272, 2293, 2387, 2406, 2488,
       2607, 2654, 2896, 2927, 2945, 2995, 3005, 3060, 3073, 3206, 3333,
       3369, 3422, 3503, 3520, 3558, 3597, 3718, 3767, 3780, 3796, 3808,
       3853, 3893, 3906, 4075, 4078, 4163, 4176, 4224, 4289, 4306, 4435,
       4497, 4500, 4575, 4601, 4639, 4690, 4740, 4807, 4823, 4880, 4890,
       4956, 5600, 5642, 5734, 5888, 5955, 5973, 5997, 6035, 6166, 6505,
       6571, 6597, 6651, 6783, 7121, 7432, 7434, 7886, 8095, 8408, 9009,
       9015, 9019, 9024, 9587, 9642, 9692, 9729, 9745, 9768, 9770, 9779,
       9792, 9811, 9839, 9879, 9888, 9905, 9982])

 Część wspólna błędów popełnionych przez RNN oraz Conv przy random_seed=1

In [167]:
common=np.nonzero(rnn-c)

In [168]:
common

(array([   2,    7,    8, ..., 9991, 9996, 9998]),)